In [2]:
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL


# TODO get these fields from dotenv
url_object = URL.create(
    drivername='postgresql+psycopg',
    host='',
    database='postgres',
    username='',
    password=''
)

engine = create_engine(url_object)





In [3]:
query = text("""
            SELECT *
            FROM liftinglookup.meet
             """)

with engine.connect() as conn:
    res = conn.execute(query)
    for row in res:
        print(row)

('Gunther', "Gunther's Test Meet", datetime.date(2024, 1, 1))


In [4]:
from lookup import LiftingCast

liftingcast = LiftingCast()


In [5]:

liftingcast.fetch_meets()


In [6]:
from multiprocessing import Pool
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from tqdm import tqdm


def fetch_lifters_from_meet(meet):
    print(f"fetching lifters for {meet['name']}")
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    driver = webdriver.Chrome(
        options=options, service=ChromeService(ChromeDriverManager().install())
    )
    driver.implicitly_wait(2)
    meet_url = f"https://liftingcast.com/meets/{meet['meet_id']}/roster"
    driver.get(meet_url)
    lifters = driver.find_elements(By.TAG_NAME, "a")
    result = []
    for lifter in lifters:
        lifter_id = lifter.get_attribute("href").split("/")[-1]
        result.append(                   
            {
                "name": lifter.text,
                "lifter_id": lifter_id,
                "meet": meet["name"],
                "meet_id": meet["meet_id"],
            }
        )     
    return result

if __name__ == "__main__":
    pool = Pool(5)
    for _ in tqdm(pool.map(fetch_lifters_from_meet, liftingcast.meets[:10])):
        pass



In [9]:
liftingcast.fetch_lifters()


100%|██████████| 526/526 [33:13<00:00,  3.79s/it]  


In [12]:
l = liftingcast.lifters

In [6]:
# liftingcast.fetch_lifter_details()


In [17]:

lifter_lists = [meet['Lifters'] for meet in liftingcast.meets if 'Lifters' in meet]
lifters = [item for sub_list in lifter_lists for item in sub_list]



In [8]:
import boto3
from botocore.config import Config
from dotenv import load_dotenv
load_dotenv()
dynamodb_config = Config(
    region_name = 'us-east-2'
)

dynamodb = boto3.resource('dynamodb', config=dynamodb_config)
lifter_table = dynamodb.Table('lifter')



In [9]:


res = lifter_table.scan()
print(res)


{'Items': [{'lifter_id': '1234', 'name': 'Gunther Kroth'}], 'Count': 1, 'ScannedCount': 1, 'ResponseMetadata': {'RequestId': '9HC0TQ4EET8OO8V74RD5U3SGRNVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Fri, 27 Oct 2023 04:31:27 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '94', 'connection': 'keep-alive', 'x-amzn-requestid': '9HC0TQ4EET8OO8V74RD5U3SGRNVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '1023172865'}, 'RetryAttempts': 0}}


In [20]:
with lifter_table.batch_writer() as bw:
    for l in lifters:
        bw.put_item(Item=l)